<article class="day-desc"><h2>--- Day 8: Playground ---</h2><p>Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad.</p>
<p>You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground!</p>
<p>Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical <a href="https://en.wikipedia.org/wiki/Junction_box" target="_blank">junction boxes</a>.</p>
<p>Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes.</p>
<p>The Elves are trying to figure out <em>which junction boxes to connect</em> so that electricity can reach <em>every</em> junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input).</p>
<p>For example:</p>
<pre><code>162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
</code></pre>
<p>This list describes the position of 20 junction boxes, one per line. Each position is given as <code>X,Y,Z</code> coordinates. So, the first junction box in the list is at <code>X=162</code>, <code>Y=817</code>, <code>Z=812</code>.</p>
<p>To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are <em>as close together as possible</em> according to <a href="https://en.wikipedia.org/wiki/Euclidean_distance" target="_blank">straight-line distance</a>. In this example, the two junction boxes which are closest together are <code>162,817,812</code> and <code>425,690,689</code>.</p>
<p>By connecting these two junction boxes together, because electricity can flow between them, they become part of the same <em>circuit</em>. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.</p>
<p>Now, the two junction boxes which are closest together but aren't already directly connected are <code>162,817,812</code> and <code>431,825,988</code>. After connecting them, since <code>162,817,812</code> is already connected to another junction box, there is now a single circuit which contains <em>three</em> junction boxes and an additional 17 circuits which contain one junction box each.</p>
<p>The next two junction boxes to connect are <code>906,360,560</code> and <code>805,96,715</code>. After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.</p>
<p>The next two junction boxes are <code>431,825,988</code> and <code>425,690,689</code>. Because these two junction boxes were <em>already in the same circuit</em>, nothing happens!</p>
<p>This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.</p>
<p>After making the ten shortest connections, there are 11 circuits: one circuit which contains <em>5</em> junction boxes, one circuit which contains <em>4</em> junction boxes, two circuits which contain <em>2</em> junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces <code><em>40</em></code>.</p>
<p>Your list contains many junction boxes; connect together the <em>1000</em> pairs of junction boxes which are closest together. Afterward, <em>what do you get if you multiply together the sizes of the three largest circuits?</em></p>
</article>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from aocd import get_data

puzzle_input = get_data(day=8, year=2025)
puzzle_input[:20]

'60838,11455,13996\n65'

In [3]:
sample_input = """162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689"""
sample_input

'162,817,812\n57,618,57\n906,360,560\n592,479,940\n352,342,300\n466,668,158\n542,29,236\n431,825,988\n739,650,466\n52,470,668\n216,146,977\n819,987,18\n117,168,530\n805,96,715\n346,949,466\n970,615,88\n941,993,340\n862,61,35\n984,92,344\n425,690,689'

In [4]:
from dataclasses import dataclass


@dataclass(frozen=True, slots=True)
class JunctionBox:
    """Represents a junction box at a 3D position."""

    x: int
    y: int
    z: int

    @classmethod
    def from_string(cls, s: str) -> "JunctionBox":
        """Parse a junction box from 'X,Y,Z' format."""
        parts = s.strip().split(",")
        if len(parts) != 3:
            raise ValueError(f"Invalid junction box format: {s}")
        return cls(int(parts[0]), int(parts[1]), int(parts[2]))


# Test
JunctionBox.from_string("162,817,812")

JunctionBox(x=162, y=817, z=812)

In [5]:
def parse_junction_boxes(s: str) -> tuple[JunctionBox, ...]:
    """Parse all junction boxes from puzzle input."""
    return tuple(JunctionBox.from_string(line) for line in s.splitlines())


# Test
sample_boxes = parse_junction_boxes(sample_input)
len(sample_boxes), sample_boxes[:3]

(20,
 (JunctionBox(x=162, y=817, z=812),
  JunctionBox(x=57, y=618, z=57),
  JunctionBox(x=906, y=360, z=560)))

In [6]:
import math


def euclidean_distance(a: JunctionBox, b: JunctionBox) -> float:
    """Calculate straight-line distance between two junction boxes."""
    return math.sqrt((a.x - b.x) ** 2 + (a.y - b.y) ** 2 + (a.z - b.z) ** 2)


# Test - distance between first and last boxes in sample
euclidean_distance(sample_boxes[0], sample_boxes[-1])

316.90219311326956

In [7]:
from itertools import combinations


def all_pairs_by_distance(
    boxes: tuple[JunctionBox, ...],
) -> tuple[tuple[int, int, float], ...]:
    """Generate all pairs of box indices sorted by distance (closest first)."""
    pairs = tuple(
        (i, j, euclidean_distance(boxes[i], boxes[j]))
        for i, j in combinations(range(len(boxes)), 2)
    )
    return tuple(sorted(pairs, key=lambda p: p[2]))


# Test - first few closest pairs
sample_pairs = all_pairs_by_distance(sample_boxes)
sample_pairs[:5]

((0, 19, 316.90219311326956),
 (0, 7, 321.560258738545),
 (2, 13, 322.36935338211043),
 (7, 19, 328.11888089532425),
 (17, 18, 333.6555109690233))

In [8]:
# Verify the closest pair matches puzzle description
first_pair = sample_pairs[0]
sample_boxes[first_pair[0]], sample_boxes[first_pair[1]]

(JunctionBox(x=162, y=817, z=812), JunctionBox(x=425, y=690, z=689))

In [9]:
from typing import Self


@dataclass(frozen=True, slots=True)
class UnionFind:
    """Immutable Union-Find (Disjoint Set Union) data structure.

    Each element starts in its own set. Uses path compression and union by rank
    for efficiency, but returns new instances rather than mutating.
    """

    parent: tuple[int, ...]  # parent[i] = parent of element i (self if root)
    rank: tuple[int, ...]  # rank[i] = rank of element i (for union by rank)

    @classmethod
    def create(cls, n: int) -> Self:
        """Create a UnionFind with n elements, each in its own set."""
        return cls(parent=tuple(range(n)), rank=tuple(0 for _ in range(n)))

    def find(self, x: int) -> tuple[Self, int]:
        """Find root of x with path compression. Returns (new_uf, root)."""
        if self.parent[x] == x:
            return self, x
        # Path compression: recursively find root and update parent
        new_uf, root = self.find(self.parent[x])
        if new_uf.parent[x] != root:
            new_parent = tuple(
                root if i == x else new_uf.parent[i] for i in range(len(new_uf.parent))
            )
            return type(self)(parent=new_parent, rank=new_uf.rank), root
        return new_uf, root

    def union(self, x: int, y: int) -> Self:
        """Union sets containing x and y. Returns new UnionFind."""
        uf, root_x = self.find(x)
        uf, root_y = uf.find(y)

        if root_x == root_y:
            return uf  # Already in same set

        # Union by rank
        if uf.rank[root_x] < uf.rank[root_y]:
            root_x, root_y = root_y, root_x

        new_parent = tuple(
            root_x if i == root_y else uf.parent[i] for i in range(len(uf.parent))
        )

        if uf.rank[root_x] == uf.rank[root_y]:
            new_rank = tuple(
                uf.rank[i] + 1 if i == root_x else uf.rank[i]
                for i in range(len(uf.rank))
            )
        else:
            new_rank = uf.rank

        return type(self)(parent=new_parent, rank=new_rank)


# Test
uf = UnionFind.create(5)
uf

UnionFind(parent=(0, 1, 2, 3, 4), rank=(0, 0, 0, 0, 0))

In [10]:
# Test union operations
uf2 = uf.union(0, 1)
uf3 = uf2.union(2, 3)
uf4 = uf3.union(0, 2)  # Should merge the two groups
uf4

UnionFind(parent=(0, 0, 0, 2, 4), rank=(2, 0, 1, 0, 0))

In [11]:
from collections import Counter


def get_circuit_sizes(uf: UnionFind) -> tuple[int, ...]:
    """Get the sizes of all circuits, sorted in descending order."""
    # Find root for each element
    roots: list[int] = []
    current_uf = uf
    for i in range(len(uf.parent)):
        current_uf, root = current_uf.find(i)
        roots.append(root)
    # Count elements per root
    counts = Counter(roots)
    return tuple(sorted(counts.values(), reverse=True))


# Test - should show {0,1,2,3} in one group and {4} in another
get_circuit_sizes(uf4)

(4, 1)

In [12]:
from functools import reduce


def connect_pairs(
    uf: UnionFind, pairs: tuple[tuple[int, int, float], ...], n: int
) -> UnionFind:
    """Connect the first n pairs of junction boxes."""
    return reduce(lambda u, p: u.union(p[0], p[1]), pairs[:n], uf)


# Test - connect first 10 pairs on sample
sample_uf = UnionFind.create(len(sample_boxes))
sample_uf_10 = connect_pairs(sample_uf, sample_pairs, 10)
get_circuit_sizes(sample_uf_10)

(5, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1)

In [13]:
def product_of_top_n(sizes: tuple[int, ...], n: int) -> int:
    """Multiply together the top n circuit sizes."""
    return reduce(lambda a, b: a * b, sizes[:n], 1)


# Test - should be 5 * 4 * 2 = 40
sample_sizes = get_circuit_sizes(sample_uf_10)
product_of_top_n(sample_sizes, 3)

40

In [14]:
def solve_part1(input_str: str, num_connections: int) -> int:
    """Solve Part 1: connect pairs and return product of top 3 circuit sizes."""
    boxes = parse_junction_boxes(input_str)
    pairs = all_pairs_by_distance(boxes)
    uf = UnionFind.create(len(boxes))
    uf_connected = connect_pairs(uf, pairs, num_connections)
    sizes = get_circuit_sizes(uf_connected)
    return product_of_top_n(sizes, 3)


# Validate sample
sample_part1 = solve_part1(sample_input, 10)
assert sample_part1 == 40
sample_part1

40

In [15]:
part1 = solve_part1(puzzle_input, 1000)
part1

47040

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Elves were right; they <em>definitely</em> don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in <em>one large circuit</em>.</p>
<p>Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at <code>216,146,977</code> and <code>117,168,530</code>. The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes (<code>216</code> and <code>117</code>) produces <code><em>25272</em></code>.</p>
<p>Continue connecting the closest unconnected pairs of junction boxes together until they're <span title="I strongly recommend making an interactive visualizer for this one; it reminds me a lot of maps from futuristic space games.">all in the same circuit</span>. <em>What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?</em></p>
</article>

In [16]:
class MutableUnionFind:
    """Mutable Union-Find for performance-critical operations.

    Isolated imperative implementation per coding guidelines:
    union-find with path compression is inherently stateful.
    """

    __slots__ = ("parent", "rank", "_num_sets")

    def __init__(self, n: int) -> None:
        self.parent = list(range(n))
        self.rank = [0] * n
        self._num_sets = n

    def find(self, x: int) -> int:
        """Find root with path compression."""
        root = x
        while self.parent[root] != root:
            root = self.parent[root]
        # Path compression
        while self.parent[x] != root:
            next_x = self.parent[x]
            self.parent[x] = root
            x = next_x
        return root

    def union(self, x: int, y: int) -> bool:
        """Union sets containing x and y. Returns True if newly connected."""
        root_x, root_y = self.find(x), self.find(y)
        if root_x == root_y:
            return False
        # Union by rank
        if self.rank[root_x] < self.rank[root_y]:
            root_x, root_y = root_y, root_x
        self.parent[root_y] = root_x
        if self.rank[root_x] == self.rank[root_y]:
            self.rank[root_x] += 1
        self._num_sets -= 1
        return True

    @property
    def num_sets(self) -> int:
        """Number of disjoint sets."""
        return self._num_sets


# Test
muf = MutableUnionFind(5)
muf.union(0, 1)
muf.union(2, 3)
muf.num_sets  # Should be 3

3

In [17]:
def find_unifying_connection(
    boxes: tuple[JunctionBox, ...],
    pairs: tuple[tuple[int, int, float], ...],
) -> tuple[int, int]:
    """Find the pair indices that forms the final single circuit (optimized)."""
    uf = MutableUnionFind(len(boxes))
    for i, j, _ in pairs:
        if uf.union(i, j) and uf.num_sets == 1:
            return (i, j)
    raise ValueError("No unifying connection found")


# Test on sample - should find connection between 216,146,977 and 117,168,530
sample_unifying = find_unifying_connection(sample_boxes, sample_pairs)
sample_boxes[sample_unifying[0]], sample_boxes[sample_unifying[1]]

(JunctionBox(x=216, y=146, z=977), JunctionBox(x=117, y=168, z=530))

In [18]:
def solve_part2(input_str: str) -> int:
    """Solve Part 2: find product of X coords of last unifying connection."""
    boxes = parse_junction_boxes(input_str)
    pairs = all_pairs_by_distance(boxes)
    i, j = find_unifying_connection(boxes, pairs)
    return boxes[i].x * boxes[j].x


# Validate sample - should be 216 * 117 = 25272
sample_part2 = solve_part2(sample_input)
assert sample_part2 == 25272
sample_part2

25272

In [19]:
part2 = solve_part2(puzzle_input)
part2

4884971896